In [1]:
#Imporations libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import ensemble
from datetime import datetime
import scipy.stats as stats

In [2]:
sns.set( # Styles Seaborn
    style='whitegrid',
    context='notebook',
    palette='Paired',
    rc={'figure.figsize':(8,4)})

In [3]:
# Affichage des nombres : séparateur des milliers, et réduction du nombre de décimales
pd.options.display.float_format = '{:,.2f}'.format

# Exploration
On dispose de 3 jeux de données : clients, produits et transactions.

In [8]:
# Les donnes sont stocker dans un repertoir source Githube Repository
url = 'https://raw.githubusercontent.com/bbouya/Data_Analyst/main/analyse_vente/sources/'
# Dataset clients
clients = pd.read_csv(url + 'customers.csv')
# Dataset produits
produits = pd.read_csv(url + 'products.csv')
# Transactions dataset
transactions = pd.read_csv(url + 'transactions.csv')


In [9]:
transactions.head()

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242


On verifier que la table ne contient aucun doublon avant de pouvoir determiner la cle primaire,
a savoir 'client_id: les 8622 clients sont tous distincts

In [11]:
#savoir nembre de doublons dn la dataframe

def doublons(df):
    print(len(df) - len(df.drop_duplicates()), 'doublons')

doublons(clients)

0 doublons


In [16]:
def cle_primaire(df_cle): # Vérification de la clé primaire
    table_length = len(df_cle) - len(df_cle.drop_duplicates())
    if table_length == 0:
        print('Clé primaire (0 doublon)')
    else:
        print('Pas une clé primaire (', table_length, 'doublons )')
    

print(cle_primaire(clients['birth']))
print(cle_primaire(clients['client_id']))

Pas une clé primaire ( 8547 doublons )
None
Clé primaire (0 doublon)
None


In [17]:
clients['client_id']

0       c_4410
1       c_7839
2       c_1699
3       c_5961
4       c_5320
         ...  
8618    c_7920
8619    c_7403
8620    c_5119
8621    c_5643
8622      c_84
Name: client_id, Length: 8623, dtype: object

In [18]:
cle_primaire(clients['client_id'])

Clé primaire (0 doublon)


In [19]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [20]:
clients.describe(include = 'all')

,client_id,sex,birth
count,8623,8623,"8,623.00"
unique,8623,2,NaN
top,c_7770,f,NaN
freq,1,4491,NaN
mean,NaN,NaN,"1,978.28"
std,NaN,NaN,16.92
min,NaN,NaN,"1,929.00"
25%,NaN,NaN,"1,966.00"
50%,NaN,NaN,"1,979.00"
75%,NaN,NaN,"1,992.00"


In [21]:
#le variable 'sex' n'affiche elle que deux modalites:
print('Modalite de sex', list(clients['sex'].unique()))

Modalite de sex ['f', 'm']


# Produits
la table produit detaille l'identifiant, le prix et la categorie de 3287 livres


In [22]:
produits.head()


,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [23]:
# la encore aucun doublon ni valuer manquante.
doublons(produits)

0 doublons


In [24]:
cle_primaire(produits["id_prod"])

Clé primaire (0 doublon)


In [25]:
produits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [45]:
produits.describe(include = "all")

,id_prod,price,categ
count,3287,"3,287.00","3,287.00"
unique,3287,NaN,NaN
top,0_2003,NaN,NaN
freq,1,NaN,NaN
mean,NaN,21.86,0.37
std,NaN,29.85,0.62
min,NaN,-1.00,0.00
25%,NaN,6.99,0.00
50%,NaN,13.06,0.00
75%,NaN,22.99,1.00


In [50]:
produits.sort_values(by='price')

,id_prod,price,categ
731,T_0,-1.00,0
2355,0_202,0.62,0
2272,0_528,0.62,0
370,0_120,0.66,0
1211,0_1844,0.77,0
...,...,...,...
2779,2_30,233.54,2
1436,2_167,236.99,2
394,2_158,247.22,2
724,2_76,254.44,2


In [51]:
print("Modalite de categ : ", list(produits['categ'].unique()))

Modalite de categ :  [0, 1, 2]


# Transactions
Le jeu des transactions detaille la date et les identifiants de la session du client et du produit vendu

In [54]:
transactions.head()

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242


Cette fois-ci la table comptes 126 

In [55]:
doublons(transactions)

126 doublons


In [56]:
doublons(transactions)

126 doublons


In [58]:
transactions.drop_duplicates(inplace = True)#Supression de duplicate
doublons(transactions)

0 doublons


In [61]:
cle_primaire(transactions[['client_id', 'date']])#cette fois ci on a deux cle primaire pas un seul

Cle primaire (0 doubon)


In [86]:
print(keys_1['_merge'].unique())
print(keys_2['_merge'].unique())

['both', 'left_only']
Categories (2, object): ['both', 'left_only']
['both', 'left_only', 'right_only']
Categories (3, object): ['both', 'left_only', 'right_only']


In [85]:
keys_1 = clients.merge( # Clés de df2 non présentes dans df1
        transactions, how='outer', indicator=True)
keys_2 = produits.merge(transactions, how = 'outer', indicator = True)
print(clients['client_id'].shape)
print(transactions['client_id'].shape)
print(keys_1['client_id'].shape)

(8623,)
(336890,)
(336911,)


In [77]:
print(produits['id_prod'].shape)
print(transactions['id_prod'].shape)
print(keys_1['id_prod'].shape)
print(transactions['id_prod'].shape[0]+21)

(3287,)
(336890,)
(336911,)
336911


In [78]:
#Comparaison d'un cle unique entre 2 dataframes

def compare_keys(variable, df1,df1_name, df2, df2_name):
    df1_keys = pd.DataFrame(df1[variable].unique())
    df2_keys = pd.DataFrame(df2[variable].unique())
    keys_1 = df1_keys.merge(#cles de df2 non presentes dans df1
            df2_keys, how = 'outer', indicator = True).loc[lambda x: x['_merge']=='right_only']
    print('Clés de', df2_name, 'non présentes dans', df1_name + ' : ', len(keys_1))   
    
    keys_2 = df2_keys.merge(  # Clés de df1 non présentes dans df2
    df1_keys, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']
    print('Clés de', df1_name, 'non présentes dans', df2_name + ' : ', len(keys_2))
    

In [79]:
compare_keys('client_id', clients, 'clients', transactions, 'transactions') # Comparaison de 'client_id' entre transactions et clients
print('- ' * 24)
compare_keys('id_prod', produits, 'produits', transactions, 'transactions') # Comparaison de la clé 'id_prod' entre transactions et produits


Clés de transactions non présentes dans clients :  0
Clés de clients non présentes dans transactions :  21
- - - - - - - - - - - - - - - - - - - - - - - - 
Clés de transactions non présentes dans produits :  1
Clés de produits non présentes dans transactions :  22


In [88]:
# cle de id_prod uniquement presente dans transactions:
transactions.drop(transactions[transactions['id_prod'].isin(produits['id_prod'])].index) 


,id_prod,date,session_id,client_id
6235,0_2245,2021-06-17 03:03:12.668129,s_49705,c_1533
10802,0_2245,2021-06-16 05:53:01.627491,s_49323,c_7954
14051,0_2245,2021-11-24 17:35:59.911427,s_124474,c_5120
17486,0_2245,2022-02-28 18:08:49.875709,s_172304,c_4964
21078,0_2245,2021-03-01 00:09:29.301897,s_3,c_580
...,...,...,...,...
322710,0_2245,2021-04-06 19:59:19.462288,s_16936,c_4167
329417,0_2245,2021-03-30 23:29:02.347672,s_13738,c_7790
330490,0_2245,2021-12-03 14:14:40.444177,s_128815,c_6189
335531,0_2245,2021-04-27 18:58:47.703374,s_26624,c_1595


,id_prod,date,session_id,client_id
6235,0_2245,2021-06-17 03:03:12.668129,s_49705,c_1533
10802,0_2245,2021-06-16 05:53:01.627491,s_49323,c_7954
14051,0_2245,2021-11-24 17:35:59.911427,s_124474,c_5120
17486,0_2245,2022-02-28 18:08:49.875709,s_172304,c_4964
21078,0_2245,2021-03-01 00:09:29.301897,s_3,c_580
...,...,...,...,...
322710,0_2245,2021-04-06 19:59:19.462288,s_16936,c_4167
329417,0_2245,2021-03-30 23:29:02.347672,s_13738,c_7790
330490,0_2245,2021-12-03 14:14:40.444177,s_128815,c_6189
335531,0_2245,2021-04-27 18:58:47.703374,s_26624,c_1595


In [111]:
transactions.describe(include='all') # Indicateurs statistiques

,id_prod,date,session_id,client_id
count,336890,336890,336890,336890
unique,3266,336855,169195,8602
top,1_369,test_2021-03-01 02:30:02.237430,s_0,c_1609
freq,1081,2,74,12855


La table ne contient aucune valeur manquante, mais comme aperçu dans les indicateurs, la variable 'date' affiche 74 valeurs aberrantes ayant pour préfixe test_. Ces 74 individus concernent le produit au prix négatif (T_0), ainsi que des identifiants de session et de clients uniques là aussi.

In [112]:
transactions_test = transactions.sort_values(by='date', ascending=False).head(75).reset_index() # Les 75 premiers individus triés par date
transactions_test

,index,id_prod,date,session_id,client_id
0,38779,T_0,test_2021-03-01 02:30:02.237450,s_0,ct_0
1,203239,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0
2,245346,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1
3,317345,T_0,test_2021-03-01 02:30:02.237448,s_0,ct_0
4,26814,T_0,test_2021-03-01 02:30:02.237447,s_0,ct_1
...,...,...,...,...,...
70,57755,T_0,test_2021-03-01 02:30:02.237413,s_0,ct_1
71,59043,T_0,test_2021-03-01 02:30:02.237413,s_0,ct_0
72,133443,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_0
73,22347,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1
